## Querying NYC resturaunt inspection data using SQL

Notebook showing the process of querying NYC resturaunt inspection data using SQL. Two examples are given, one to group resturaunts by borough and show average score per borough, and one to group by cuisine type and show the number of violations per cuisine type within a given time period

In [6]:
import pandas as pd
import sqlite3

In [7]:
%load_ext sql
%sql sqlite:///information_schema.db
we = pd.read_csv('WebExtract.csv', encoding='latin-1')
action = pd.read_csv('Action.csv', encoding='latin-1')
cuisine = pd.read_csv('Cuisine.csv', encoding='latin-1')
violation = pd.read_csv('Violation.csv', encoding='latin-1', on_bad_lines='skip')

/var/folders/n8/mc27w99j19q1kx20z133q0w40000gp/T/ipykernel_37473/3527669267.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  we = pd.read_csv('WebExtract.csv', encoding='latin-1')


In [8]:
cnn = sqlite3.connect('information_schema.db')

In [9]:
'''we.to_sql('webextract', cnn)
action.to_sql('action', cnn)
cuisine.to_sql('cuisine', cnn)
violation.to_sql('violation', cnn)'''

"we.to_sql('webextract', cnn)\naction.to_sql('action', cnn)\ncuisine.to_sql('cuisine', cnn)\nviolation.to_sql('violation', cnn)"

Remove bad entries from data set, note CAMIS is the specific resturaunt ID

In [10]:
%%sql

DROP TABLE if EXISTS good_dates;

CREATE TEMP TABLE good_dates AS

with cols as (select CAMIS, INSPDATE
              from webextract
              GROUP BY CAMIS, INSPDATE
              having count(SCORE)>0)


SELECT webextract.CAMIS, webextract.INSPDATE, webextract.SCORE, webextract.ZIPCODE, webextract.BORO
FROM webextract JOIN cols 
ON webextract.CAMIS = cols.CAMIS 
AND webextract.INSPDATE = cols.INSPDATE

 * sqlite:///information_schema.db
Done.
Done.


[]

number of entries removed

In [11]:
%%sql

SELECT COUNT(*) FROM (SELECT CAMIS, INSPDATE, COUNT(SCORE)
FROM webextract
GROUP BY CAMIS, INSPDATE
HAVING COUNT(SCORE) = 0) as df

 * sqlite:///information_schema.db
Done.


COUNT(*)
8255


## Next I will query the average score per burough

Using the most recent score for each resturaunt

In [12]:
%%sql 

DROP TABLE if EXISTS boro1;

CREATE TEMP TABLE boro1 AS

SELECT BORO, CAMIS, MAX(INSPDATE) as max_date
FROM good_dates
GROUP BY CAMIS

 * sqlite:///information_schema.db
Done.
Done.


[]

Join borough data with resturaunt score from webextract table

In [13]:
%%sql 

DROP TABLE if EXISTS add_scores;

CREATE TEMP TABLE add_scores AS

SELECT b.BORO, b.CAMIS, b.max_date, we.SCORE
FROM boro1 as b JOIN webextract as we
ON we.CAMIS = b.CAMIS and we.INSPDATE = b.max_date

 * sqlite:///information_schema.db
Done.
Done.


[]

Average over score based on borough camis and max date grouping

In [14]:
%%sql 

DROP TABLE if EXISTS score_avg;

CREATE TEMP TABLE score_avg AS

SELECT BORO, CAMIS, max_date, AVG(SCORE) as avgscore
FROM add_scores
GROUP BY BORO, CAMIS, max_date

 * sqlite:///information_schema.db
Done.
Done.


[]

group into boroughs and only keep resturaunts with over 100 reviews

In [15]:
%%sql

DROP TABLE if EXISTS borugh;

CREATE TEMP TABLE borugh AS

SELECT BORO, AVG(avgscore) AS avg_score, COUNT(DISTINCT CAMIS) AS num_resturaunts
FROM score_avg 
GROUP BY BORO
HAVING num_resturaunts > 100
ORDER BY avg_score

 * sqlite:///information_schema.db
Done.
Done.


[]

In [16]:
%%sql

SELECT * FROM borugh
ORDER BY avg_score

 * sqlite:///information_schema.db
Done.


BORO,avg_score,num_resturaunts
2,10.776852257781675,2281
1,11.48963704377717,9891
3,11.596854163104805,5849
4,11.82259553258261,5417
5,11.899122807017545,912


In [17]:
data = _
df = data.DataFrame()
df.num_resturaunts.sum()

24350

In [18]:
df['BORO'][0] = 'THE BRONX'
df['BORO'][1] = 'MANHATTAN'
df['BORO'][2] = 'BROOKLYN'
df['BORO'][3] = 'QUEENS'
df['BORO'][4] = 'STATEN ISLAND'

/var/folders/n8/mc27w99j19q1kx20z133q0w40000gp/T/ipykernel_37473/4166052207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BORO'][0] = 'THE BRONX'


In [19]:
records = df.to_records(index=False)
result = list(records)

In [20]:
result

[('THE BRONX', 10.77685226, 2281),
 ('MANHATTAN', 11.48963704, 9891),
 ('BROOKLYN', 11.59685416, 5849),
 ('QUEENS', 11.82259553, 5417),
 ('STATEN ISLAND', 11.89912281, 912)]

## Next I am going to determine the number of violations for a given cuisine

In [21]:
%%sql

select * from webextract
limit 1

 * sqlite:///information_schema.db
Done.


index,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE,10462.0,7188924968,8,2014-03-03 00:00:00,D,10F,2.0,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000


In [22]:
%%sql

select * from cuisine
limit 1

 * sqlite:///information_schema.db
Done.


index,CUISINECODE,CODEDESC
0,2,African


again remove null entries

In [23]:
%%sql

DROP TABLE if EXISTS table1;

CREATE TEMP TABLE table1 AS

with cols as (select CAMIS, INSPDATE
              from webextract
              GROUP BY CAMIS, INSPDATE
              having count(SCORE)>0)


SELECT we.CAMIS, we.INSPDATE, we.VIOLCODE, we.CUISINECODE
FROM webextract as we JOIN cols as c
ON we.CAMIS = c.CAMIS 
AND we.INSPDATE = c.INSPDATE

 * sqlite:///information_schema.db
Done.
Done.


[]

Next few cells are just exploring the data

In [24]:
%%sql

select * from table1
order by cuisinecode DESC
limit 10

 * sqlite:///information_schema.db
Done.


CAMIS,INSPDATE,VIOLCODE,CUISINECODE
40384528,2014-06-30 00:00:00,04M,99
40384528,2014-06-30 00:00:00,08A,99
40384528,2014-06-30 00:00:00,06E,99
40384528,2014-06-30 00:00:00,10F,99
40384528,2014-06-30 00:00:00,06C,99
40392141,2014-04-21 00:00:00,04L,99
40392141,2014-04-21 00:00:00,08A,99
40392141,2014-04-21 00:00:00,10H,99
40678067,2014-02-22 00:00:00,04L,99
40678067,2014-02-22 00:00:00,08A,99


In [25]:
%%sql

select * from table1
WHERE camis = 30075445
group by CAMIS, INSPDATE, VIOLCODE
ORDER BY CAMIS, INSPDATE
limit 10

 * sqlite:///information_schema.db
Done.


CAMIS,INSPDATE,VIOLCODE,CUISINECODE
30075445,2011-03-10 00:00:00,04L,8
30075445,2011-03-10 00:00:00,10F,8
30075445,2011-03-10 00:00:00,10I,8
30075445,2011-03-10 00:00:00,16A,8
30075445,2011-11-23 00:00:00,06C,8
30075445,2011-11-23 00:00:00,10B,8
30075445,2011-11-23 00:00:00,10H,8
30075445,2012-12-31 00:00:00,02G,8
30075445,2012-12-31 00:00:00,04L,8
30075445,2012-12-31 00:00:00,06C,8


Given that most restaurants undergo multiple inspections, each potentially revealing multiple violations, or may experience extended periods without any violations

In order to address this normalization I will calculate a violation rate by dividing the total number of violations by the duration (in years) the restaurant has been in operation. To estimate this duration, I will determine the time difference between the oldest and newest inspection dates. For instances where the duration is less than 30 days, we will assume it to be 30 days to account for restaurants inspected only once.

In [26]:
%%sql 

DROP TABLE if EXISTS cuisine1;

CREATE TEMP TABLE cuisine1 AS

SELECT CAMIS, CUISINECODE, (JULIANDAY(MAX(INSPDATE))-JULIANDAY(MIN(INSPDATE))) as resturaunt_days, COUNT(1) as num_violations
FROM table1
GROUP BY CAMIS

 * sqlite:///information_schema.db
Done.
Done.


[]

In [27]:
%%sql 

UPDATE cuisine1
SET resturaunt_days = 30
WHERE resturaunt_days < 30;

 * sqlite:///information_schema.db
2260 rows affected.


[]

In [28]:
%%sql 

DROP TABLE if EXISTS cuisine2;

CREATE TEMP TABLE cuisine2 AS

SELECT CUISINECODE, (SUM(resturaunt_days)/365) as total_years, SUM(num_violations) as total_violations
FROM cuisine1
GROUP BY CUISINECODE
HAVING total_violations > 100

 * sqlite:///information_schema.db
Done.
Done.


[]

In [29]:
%%sql 

DROP TABLE if EXISTS cuisine3;

CREATE TEMP TABLE cuisine3 AS

select c.CODEDESC, (c2.total_violations/c2.total_years) as reports_per_year
FROM cuisine2 as c2 JOIN cuisine as c
ON c.CUISINECODE = c2.CUISINECODE

 * sqlite:///information_schema.db
Done.
Done.


[]

In [30]:
%%sql

select * from cuisine3
ORDER BY reports_per_year

 * sqlite:///information_schema.db
Done.


CODEDESC,reports_per_year
Hotdogs/Pretzels,4.29350164969158
Soups & Sandwiches,6.01755371436596
Donuts,6.166799725218618
"Ice Cream, Gelato, Yogurt, Ices",6.337295068616821
Hotdogs,6.453558961906207
Sandwiches,6.661998352034242
Café/Coffee/Tea,6.7971463492951765
"Bottled beverages, including water, sodas, juices, etc.",7.151427703523694
"Juice, Smoothies, Fruit Salads",7.206375961890802
Not Listed/Not Applicable,7.443268935909231
